# Getting Started with using Anthropic models using Amazon Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio on ml.t3.medium instance*

---

In this demo notebook, we demonstrate how to use the boto3 Python SDK to work with Amazon Bedrock Foundation Models. If you are running this in AWS provided accounts, excessive API calls to Bedrock APIs may results in throttling and your account may get blocked

---

You can now access Claude 2, the latest version of Anthropic’s large language model (LLM), on Amazon Bedrock. Claude 2 can take up to 200,000 tokens in each prompt, meaning it can work over hundreds of pages of text, or even an entire book. Claude 2 can also write longer documents—on the order of a few thousand tokens—compared to its prior version, giving you even greater ways to develop generative AI applications using Amazon Bedrock.

Anthropic, an AI safety and research lab that builds reliable, interpretable, and steerable AI systems, is the maker of the state-of-the art LLM, Claude. The new version of the LLM, Claude 2, can process large amounts of text. With its 100,000 token context window, which is equivalent to about 200 pages of information, Claude 2 lets you upload large amounts of data. As a result, you can use documents, emails, FAQs, chat transcripts or even entire codebases as inputs for Claude 2 to edit, rewrite, summarize, produce Q&A content, generate code, and more.

## Imports and set up

In [ ]:
! pip install pypdf --quiet

## Set Up Bedrock Client

To help with this, we've provided a get_bedrock_client() utility method that supports passing in different options. You can find the implementation in ../utils/bedrock.py

The get_bedrock_client() method accepts runtime (default=True) parameter to return either bedrock or bedrock-runtime client.

In [ ]:

import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)


## The Great Gatsby

In [ ]:

from pypdf import PdfReader

reader = PdfReader("./data/the-great-gatsby.pdf")
number_of_pages = len(reader.pages)
text = ''.join([page.extract_text() for page in reader.pages])
print(text[:1000])


In [ ]:

total_no_of_words_input = len(text.split())
# total_no_of_tokens = total_no_of_words/0.75
print(f"Total Number of Pages in Great Gatsby Book: {number_of_pages}")
print(f"Total Number of words in Great Gatsby Book: {total_no_of_words_input}")
# print(f"Total Number of tokens in Great Gatsby Book: {total_no_of_tokens}")


## Let's get create a prompt and invoke Claude Instant model

In [ ]:

# If you'd like to try your own prompt, edit this parameter!
prompt_data = f"""Here is The Great Gatsby Book: <report>{text[:5000]}</report>

I would like to know the list all the characters from this book. Describe the main characters and their roles in this book

"""

body = json.dumps({
    "max_tokens": 1024,
    "messages": [{"role": "user", "content": prompt_data}],
    "anthropic_version": "bedrock-2023-05-31"
})

modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"


In [ ]:

import time

# get the start time
st = time.time()

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)

et = time.time()

elapsed_time = et - st

print('Execution time:', elapsed_time, 'seconds')


response_body = json.loads(response.get("body").read())
response_completion = response_body.get("content")[0]["text"]
print(response_completion)


### On-Demand Pricing calcualtion

Please refer https://aws.amazon.com/bedrock/pricing/ for latest on-demand provisioned througput pricing details

In [ ]:

total_no_words_output = len(response_completion.split())
 
total_no_of_tokens_input = response['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-input-token-count']
total_no_tokens_output = response['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-output-token-count']
total_response_time = response['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-invocation-latency']



print(f"Total Number of words in Great Gatsby Book: {total_no_of_words_input}")
print(f"Total Number of tokens in Great Gatsby Book: {total_no_of_tokens_input}")
print(f"Total Number of words in Output: {total_no_words_output}")
print(f"Total Number of tokens in Output: {total_no_tokens_output}")

print('Execution time:', int(total_response_time)/1000, 'seconds')

cost_of_input_tokens = int(total_no_of_tokens_input) * 0.0000008 #on-demand pricing per input token
cost_of_output_tokens = int(total_no_tokens_output) * 0.0000024 #on-demand pricing per output token
total_cost = cost_of_input_tokens + cost_of_output_tokens

print(f"Total cost for input tokens: {cost_of_input_tokens}")
print(f"Total cost of output tokens: {cost_of_output_tokens}")
print(f"Total cost: {total_cost}")


### Let's load Amazon's annual report and analyze the document

In [ ]:

from pypdf import PdfReader

amazon_reader = PdfReader("./data/Amazon-2022-Annual-Report.pdf")
amazon_number_of_pages = len(amazon_reader.pages)
amazon_text = ''.join([page.extract_text() for page in amazon_reader.pages])
amazon_text = amazon_text[:50000] # for demonstration purposes
print(amazon_text[:1000])


## Set up prompt template with following best practices

1. Include formatting
    - Mandatory prompt formats for Claude: We are requiring all Amazon Bedrock customers consuming Anthropic’s Claude to send prompts to the model in the following syntax: "<Customer Prompt>". If this syntax is not accounted for, model invocation requests will fail with the following error message: "ValidationException Input validation failed. Check your request parameters and retry the request. HTTP Status Code: 400”. No other change is required to prompts and content being sent through the Amazon Bedrock API for Anthropic’s Claude models.

    2. Being clear and direct

    3. Assigning roles (aka role prompting)

    4. Using XML tags

    5. Separating data from instructions

    6. Thinking step by step

In [ ]:

# If you'd like to try your own prompt, edit this parameter!
prompt_data = f"""Here is Amazon financial annual report: <report>{amazon_text}</report>

You are a useful financial analyst. Please do the following:
1. Summarize the annual report and provide highlights in bullet points. (In <Annual_Report_Summary> tags.)
2. What is the total value creation in 2022?
3. What is the total value creation in 2020?
4. How was year 2020 different from 2022 for the shareholders?

Lets think step by step

"""


### Invoke Claude V2 model using Bedrock API

The Anthropic Claude models support the following parameters to control the length of the generated response.

Maximum length (max_tokens_to_sample) – Specify the maximum number of tokens to use in the generated response.

Stop sequences (stop_sequences) – Configure up to four sequences that the model recognizes. After a stop sequence, the model stops generating further tokens. The returned text doesn't contain the stop sequence.

In [ ]:

body = json.dumps({
    "max_tokens": 1024,
    "messages": [{"role": "user", "content": prompt_data}],
    "anthropic_version": "bedrock-2023-05-31"
})

modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("content")[0]["text"])


## Prompt guidance for different NLP tasks

### Information Extraction


In [ ]:

# If you'd like to try your own prompt, edit this parameter!
information_extraction_template = f"""Here is Amazon financial annual report: <report>{amazon_text}</report>

You are a useful financial analyst. Please precisely extract information about executive officers. I am looking to extract following data points in a JSON format. If you are not able to extract any of the information, write 'NA'. Do not say anything else.

1. Name of the executive officer
2. Age
3. Position held
4. Desctiption about each of the officer


Lets think step by step

"""


In [ ]:

body = json.dumps({
    "max_tokens": 1024,
    "messages": [{"role": "user", "content": information_extraction_template}],
    "anthropic_version": "bedrock-2023-05-31"
})


modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("content")[0]["text"])


### PII Extraction

In [ ]:

pii_extraction_template = f"""Here is some text. We want to remove all personally identifying information from this text and replace it with ****.  It's very important that names, phone numbers, and email addresses, gets replaced with ******.
Here is the text, inside <text></text> XML tags

<text>
Phone Directory:
John Latrabe, 800-232-1995,  john909709@geemail.com
Josie Lana, 800-759-2905,   josie@josielananier.com
Keven Stevens, 800-980-7000,  drkevin22@geemail.com

Phone directory will be kept up to date by the HR manager.
</text>

Please put your sanitized version of the text with PII removed in <response></response> XML tags

"""


In [ ]:

body = json.dumps({
    "max_tokens": 1024,
    "messages": [{"role": "user", "content": pii_extraction_template}],
    "anthropic_version": "bedrock-2023-05-31"
})

modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("content")[0]["text"])


## One Up on Wall Street

### Challenges with reading PDF docs

Certain PDFs are encoding differently, usage of packages like PyPDF does not guaratee text extraction from PDFs. You may have to adhere to robust preprocessing practices while working with different doc types. Below is an example where PDF extraction doesnt yeild the desired text.

In [ ]:

one_up_reader = PdfReader("./data/one-up-on-wall-street-full.pdf")
one_up_number_of_pages = len(one_up_reader.pages)
one_up_text = ''.join([page.extract_text() for page in one_up_reader.pages])
#one_up_text = one_up_text[:50000]
print(one_up_text)


In [ ]:

# If you'd like to try your own prompt, edit this parameter!
one_up_data = f"""Here is book about Mr. Peter Lynch: <book>{one_up_text}</book>

You are a useful financial analyst. Please answer the question from the book only if you can find relevant context is present, if you dont have the context from the book, Say I dont know:
According to the author of the above book, what is the most important factor driving long-term value of a stock?

"""

In [ ]:
body = json.dumps({
    "max_tokens": 1024,
    "messages": [{"role": "user", "content": one_up_data}],
    "anthropic_version": "bedrock-2023-05-31"
})

modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("content")[0]["text"])

## Conclusion

In this notebook, you got familiar with
1. Working with Claude models 
2. How to leverage Claude's long context length and usecases where this might be a best fit
3. Best practices and recommendations while using Claude models
4. Leveraging Claude in different NLP tasks using promptig